# **Proyecto 1 - Etapa I**

Integrantes:
* Santiago Rodriguez Bernal - 202011182
* Juan Camilo Pacheco Carvajal - XXXXXXXXX


# 1. Entendimiento de los datos

In [1]:
#Instalaciones

#Y Data Profiling

!pip install ydata-profiling

#Libreria para remplazar numeros por palabras

!pip install num2words

#Deteccion de lenguaje para eliminar entradas que no esten en español

!pip install langdetect

#Procesamientno de lenguaje natural en español

!pip install stanza



Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for ydata-profiling from https://files.pythonhosted.org/packages/49/03/04ce11264d78f7a9f1025a13f8a078aa18eb2a9c628bc0a9a9c18673062c/ydata_profiling-4.7.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=2 from https://files.pythonhosted.org/packages/e5/f3/8296f550276194a58c5500d55b19a27ae0a5a3a51ffef66710c58544b32d/pydantic-2.6.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/85.1 kB ? eta -:--:--
     -------------------------------------- 85.1/85.1 kB 959.6 kB/s eta 0:00:00
  Obtaining dependency information for visions[type_image_path]<0.7.7,>=0.7.5 from https://files.pythonhosted.org/packages/7c/bf/612b24e711ae25dea9af19b9304634b8949faa0b035fad47e8bcadf62f59/visions-0.7.6-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  P

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.6.4 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:

#Imports para procesamiento de texto

#Tokenizacion y lematizacion

import stanza

#Para integrar pasos de la limpieza adicionales

from stanza.pipeline.processor import Processor, register_processor

#Paquete español

stanza.download('es')

#Para manejo de numeros, singluares, plurarles en lenguaje

from num2words import num2words
#Deteccion de lenguaje

from langdetect import detect

from langdetect import DetectorFactory, detect_langs

# Librería Natural Language Toolkit, usada para trabajar con textos
import nltk

# Punkt permite separar un texto en frases.
nltk.download('stopwords')

from nltk.corpus import stopwords

#Operaciones con expresiones regulares y unicode
import re, string, unicodedata

2024-04-05 22:29:46 INFO: Downloaded file to C:\Users\zeify\stanza_resources\resources.json
2024-04-05 22:29:46 INFO: Downloading default packages for language: es (Spanish) ...


2024-04-05 22:32:39 INFO: Downloaded file to C:\Users\zeify\stanza_resources\es\default.zip
2024-04-05 22:32:44 INFO: Finished downloading models and saved to C:\Users\zeify\stanza_resources
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zeify\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
#Imports generales para analisis de datos y ML
import pandas as pd
import numpy as np
import sys
import statistics
from ydata_profiling import ProfileReport

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

C:\Users\zeify\AppData\Local\Temp\ipykernel_23212\122007551.py:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [4]:
# Lectura de los datos.
df_turi = pd.read_csv('tipo2_entrenamiento_estudiantes.csv', sep=',', encoding='utf-8')

df_turi.head()

,Review,Class
0,Muy buena atención y aclaración de dudas por p...,5
1,Buen hotel si están obligados a estar cerca de...,3
2,"Es un lugar muy lindo para fotografías, visite...",5
3,Abusados con la factura de alimentos siempre s...,3
4,Tuvimos un par de personas en el grupo que rea...,3


In [5]:
df_turi.dtypes

Review    object
Class      int64
dtype: object

# Impresion Reporte Y Data Profiling

In [7]:
reviews = df_turi.copy()
#Longitud del comentario (numero de caracteres)
reviews['Conteo'] = [len(x) for x in reviews['Review']]
#Palabra(s) mas frecuente(s)
reviews['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in reviews['Review']]
#Longitud de la palabra mas larga del comentario
reviews['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in reviews['Review']]
#Longitud de la palabra mas corta del comentario
reviews['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in reviews['Review']]

profile = ProfileReport(reviews, title='Profiling Report', explorative=True)

profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
#Conteo de entradas nulas
df_turi.isnull().sum()

Review    0
Class     0
dtype: int64

In [15]:
from langdetect import DetectorFactory
#Verificamos entradas en otros idiomas
DetectorFactory.seed = 42
x =df_turi[df_turi.Review.map(detect)  != 'es']

x.to_csv('output.csv', index=False)

x


,Review,Class
296,Excelente! Servicio estuvo excelente. \nLa com...,5
711,mas o menos estan mejor las de saltillo lñdkjv...,2
3749,First part of the tour was Rivera's actual hom...,2
4490,esta rico,4
6518,A few months after ciudad perdida we keep on t...,5
6869,Cementerio Antiguo. Patrimonio Mundial. facil ...,5
7455,This university is unique around the world. Th...,5


Una observacion es que la libreria no esta funcionando correctamente en su totalidad pues tenemos 3 entradas escritas en español, que fueron detectadas erroneamente como en otro idioma por lo que al hacer la eliminacion las tendremos encuenta

In [11]:
#Revisamos los registros duplicados
df_turi.duplicated().sum()



73

# 2. Preparacion de los datos

In [16]:
#Eliminamos entradas en otros idiomas
df_turi = df_turi.drop(x.index)

In [17]:
#Verificamos cuantos datos quedan
df_turi.shape

(7868, 2)

## 2.1 Limpieza, tokenizacion y lemmatizacion

In [18]:
#Remplaza los numeros por su representacion en palabras
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
#Remueve todo caracter no latino (conserva espacios y numeros)
def remove_nonlatin(words):
  new_words = []
  for word in words:
    new_word = ''
    for ch in word:
      if unicodedata.name(ch).startswith(('LATIN', 'DIGIT', 'SPACE')):
        new_word += ch
    new_words.append(new_word)
  return new_words

#Remueve palabras comunes que no aportan informacion
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    s = set(stopwords.words('spanish'))
    for word in words:
        if word not in s:
            new_words.append(word)
    return new_words

#Remueve puntuacion
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = ''
    for word in words:
            new_words += re.sub(r'[^\w\s]', ' ', word)
    return new_words

 #Procesamiento de cada review usando stanza
def tokenLemma():
  df_turi['words'] = df_turi['Review'].apply(remove_punctuation)
  #Creamos un pipeline para tokenizacion y lematizacion
  nlp = stanza.Pipeline('es', processors = 'tokenize,mwt,pos,lemma', use_gpu=True)
  in_docs = [stanza.Document([], text=d) for d in df_turi.words]
  return nlp(in_docs)

#Funcion secundaria para procesar cada token
def procesamientoPalabras(words):
    words = remove_nonlatin(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#Funcion principal para el pre-procesamiento
def preprocessing():
  out_docs = tokenLemma()
  palabras = []

  for doc in out_docs:
    reviewAct = []
    for sentence in doc.sentences:
      for word in sentence.words:
        if(word.pos != 'PUNCT' and word.pos != 'SYM'):
          reviewAct.append(word.lemma.lower())
    palabras.append(reviewAct)

  df_turi['words'] = palabras
  df_turi['words'] = df_turi['words'].apply(procesamientoPalabras)

In [19]:
# LLamamos la funcion
#Descomentar el processing si se quiere revisar, si no el resultado con los datos procesados esta en el archivo df_procesado
preprocessing()

2024-04-05 23:33:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-05 23:33:07 INFO: Downloaded file to C:\Users\zeify\stanza_resources\resources.json
2024-04-05 23:33:08 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| pos       | ancora_charlm   |
| lemma     | ancora_nocharlm |

2024-04-05 23:33:08 WARNING: GPU requested, but is not available!
2024-04-05 23:33:08 INFO: Using device: cpu
2024-04-05 23:33:08 INFO: Loading: tokenize
2024-04-05 23:33:11 INFO: Loading: mwt
2024-04-05 23:33:11 INFO: Loading: pos
2024-04-05 23:33:11 INFO: Loading: lemma
2024-04-05 23:33:11 INFO: Done loading processors!


In [ ]:

#Guardamos los datos procesados. Descomentar si se quiere crear el archivo despues de procesar los datos.
df_turi.to_csv('df_procesado.csv', index=False, sep= ",")

#Leemos los datos procesados
df_turi_Pre = pd.read_csv("df_procesado.csv", sep = ",", encoding='utf-8')
import ast
df_turi_Pre['words'] = df_turi_Pre['words'].apply(ast.literal_eval)